<a href="https://colab.research.google.com/github/stenforsm/predict-stock-news/blob/main/functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('b_news.csv')
spo = df.spo.values
price = df["close price change"].values

In [3]:
# class_plus = 1; class_minus = 0
y = np.sign(price)
y = np.where(y==1,1,0)

In [4]:
# train split
from sklearn.model_selection import train_test_split
spo_train, spo_split, y_train, y_split = train_test_split(spo, y, train_size = 0.8, random_state = 42)
# test valid split
spo_valid, spo_test, y_valid, y_test = train_test_split(spo_split, y_split, test_size = 0.5, random_state = 42)

In [5]:
# BOW text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(binary=True)
tfidf_train = tfidf.fit_transform(spo_train).astype('float16')
tfidf_valid = tfidf.transform(spo_valid).astype('float16')
tfidf_test = tfidf.transform(spo_test).astype('float16')
n = tfidf_train.shape[1] # to determine input shape
print(n)

4070


In [6]:
# count text vectorization
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
count_train = (count.fit_transform(spo_train)).toarray()
count_valid = (count.transform(spo_valid)).toarray()
count_test = (count.transform(spo_test)).toarray()
count_train.shape

(6104, 4070)

In [7]:
from tensorflow.keras import layers

tfidf_input = tf.keras.Input(shape=(n), name='tfidf')
count_input = tf.keras.Input(shape=(n), name='count')

x = layers.concatenate([tfidf_input, count_input]) # merge all available features into a single large vector via concatenation
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10)(x)

model = tf.keras.Model(inputs=[tfidf_input, count_input], outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tfidf (InputLayer)              [(None, 4070)]       0                                            
__________________________________________________________________________________________________
count (InputLayer)              [(None, 4070)]       0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 8140)         0           tfidf[0][0]                      
                                                                 count[0][0]                      
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           521024      concatenate[0][0]            

In [10]:
from tensorflow.keras import losses

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model.fit({'tfidf': tfidf_train, 'count': count_train}, y_train, validation_data= ({'tfidf': tfidf_valid, 'count': count_valid}, y_valid), callbacks=callback, epochs=20)

Epoch 1/20
191/191 [==============================] - 2s 10ms/step - loss: 0.4619 - accuracy: 0.7849 - val_loss: 0.6618 - val_accuracy: 0.6710
Epoch 2/20
191/191 [==============================] - 2s 9ms/step - loss: 0.4268 - accuracy: 0.8032 - val_loss: 0.6760 - val_accuracy: 0.6710
Epoch 3/20
191/191 [==============================] - 2s 8ms/step - loss: 0.3988 - accuracy: 0.8211 - val_loss: 0.7021 - val_accuracy: 0.6527
Epoch 4/20
191/191 [==============================] - 2s 8ms/step - loss: 0.3737 - accuracy: 0.8354 - val_loss: 0.7002 - val_accuracy: 0.6894
Epoch 5/20
191/191 [==============================] - 2s 8ms/step - loss: 0.3481 - accuracy: 0.8467 - val_loss: 0.7226 - val_accuracy: 0.6697
Epoch 6/20
191/191 [==============================] - 2s 8ms/step - loss: 0.3263 - accuracy: 0.8565 - val_loss: 0.7225 - val_accuracy: 0.6776
Epoch 7/20
191/191 [==============================] - 2s 9ms/step - loss: 0.3049 - accuracy: 0.8691 - val_loss: 0.7397 - val_accuracy: 0.6868


In [11]:
model.evaluate({'tfidf': tfidf_test, 'count': count_test}, y_test)

24/24 [==============================] - 0s 3ms/step - loss: 0.7640 - accuracy: 0.6649


[0.7640295624732971, 0.6649214625358582]

In [12]:
predictions = model.predict({'tfidf': tfidf_test, 'count': count_test})
y_predict = np.argmax(predictions, axis=1)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_predict))

from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_predict))

[[335  91]
 [165 173]]
0.31146691750601396
